In [1]:
import os
# import deepspeech
import numpy as np
import pandas as pd
from pydub import AudioSegment
import mysql.connector
import shutil
from datasets import load_dataset

/home/crow/miniconda3/envs/pluseleven/lib/python3.11/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
from all_legacy_code.src import audio_mgmt
from all_legacy_code.src import face_mgmt

pygame 2.5.2 (SDL 2.28.2, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
# Load Pre-trained Model
ds = deepspeech.Model('resources/deepspeech/pretrained/deepspeech-0.9.3-models.pbmm')

In [4]:
# Load constants
AUDIO_TRAINING_PATH = 'resources/deepspeech/audio/sahil_audio'

For training the DeepSpeech model with custom examples, we need to feed it audio files. Let us create some functions for reading and preprocessing audio files for training purposes, and storing them into database.

Let's start with creating a cursor for testing purposes.

In [5]:
conn_audio = mysql.connector.connect(
    host='localhost',
    user='root',
    password='0301sonaL',
    database='chato_audio',
    auth_plugin='mysql_native_password'
)

cursor_audio = conn_audio.cursor()

**Read all transcripts into a list**

In [6]:
with open(os.path.join('resources/deepspeech/audio', 'sahil_transcripts.txt')) as fh:
    transcripts = fh.readlines()

**Store into database**

In [7]:
transcripts

['Good evening! I would like to start with a Caesar salad, please.\n',
 'Hello there! May I have the margherita pizza, please?\n',
 'Hi! I will have the grilled chicken sandwich with a side of fries.\n',
 'Greetings! I am in the mood for some sushi. I will take the spicy tuna roll, please.\n',
 'Howdy! Could I get a classic cheeseburger with no onions, and a cola to drink?\n',
 'Hey! I would like the vegetable stir-fry, but make it extra spicy, please.\n',
 'Good afternoon! I will go for the shrimp scampi pasta, and a glass of white wine.\n',
 'Hi there! I would like the chicken quesadilla, and can I get some guacamole on the side?\n',
 'Hello! I will have the seafood chowder as a starter, and then the grilled salmon for my main course.\n',
 'Hey! I am in the mood for dessert. I will have the chocolate lava cake, please.\n',
 'Hi there! I would like to order a Whopper with cheese, large fries, and a Coke, please.\n',
 'Hello! Can I get a Bacon King Jr. and some onion rings, please?\n',

In [8]:
file = os.path.join('resources/deepspeech/audio/sahil_audio/34.m4a')
audio_mgmt.store_into_database(file, transcripts[33], cursor_audio)
conn_audio.commit()

**Read from database**

In [10]:
row = audio_mgmt.read_wav_from_database(41, cursor=cursor_audio)

In [11]:
row[0]

array([18770, 17990,  3602, ...,     0,     0,     0], dtype=int16)

**Transcribe using DeepSpeech**

In [12]:
y_pred = ds.stt(row[0])
y = row[0]
print('Actual:\t', y)
print('Predicted:\t', y_pred)

Actual:	 [18770 17990  3602 ...     0     0     0]
Predicted:	 i helo how are you


**Synthsize Text**

In [2]:
audio_mgmt.speak('That will be 4$ 45 cents, to the window please')

**Store Image into database**

In [3]:
conn_face = mysql.connector.connect(
    host='localhost',
    user='root',
    password='0301sonaL',
    database='chato_customer',
    auth_plugin='mysql_native_password'
)

cursor_face = conn_face.cursor()

In [4]:
os.chdir('/home/crow/Iota/pluseleven')

In [6]:
test_face = 'temp_faces/Sahil.jpg'
face_mgmt.store_into_database(file=test_face, cursor=cursor_face)

In [7]:
conn_face.commit()

**Test Listen**

In [6]:
speech = audio_mgmt.listen()

Listening...


In [7]:
speech

array([  0,   0,  -1, ..., -13, -39, -60], dtype=int16)

In [8]:
ds.stt(speech)

'wit up ed anyon an i de i tet to ma'

**Prepare Data**

In [4]:
# Change the indices in split parameter according to `current_i`. `current_i` represents the number of audio files written so far.
common_voice_subset = load_dataset('mozilla-foundation/common_voice_11_0', 'en', split='train[:2400]')

In [8]:
# 2400 files have been written. Change `current_i` correspondingly when calling this function.
def move_audio_files(current_i, dataset):
    for filename in dataset['path']:
        shutil.copy(file, f'{AUDIO_TRAINING_PATH}/common_voice_audio/{current_i}.mp3')
        current_i+=1

In [11]:
def write_transcripts(filename=f'{AUDIO_TRAINING_PATH}/common_voice_transcripts.txt',*, dataset):
    with open(filename, 'a') as fh:
        for line in dataset['sentence']:
            fh.write(line+'\n')